<a href="https://colab.research.google.com/github/CHAbhishek/Fantasy-team-prediction/blob/main/fantasy_team_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.kaggle.com/code/abhishekkoshta/ipl-dream11-winner-team-prediction-2021

https://www.kaggle.com/code/depekha/ipl-complete-dataset-2008-2020/notebook

https://github.com/abhishek374/dream11
https://medium.com/analytics-vidhya/dream11-team-predictor-with-python-and-machine-learning-f0dfce1489eb

Creating fantasy points

In [ ]:
# Fantasy Points

Batsman_points = {'Run':1, 'bFour':1, 'bSix':2, '30Runs':4,
        'Half_century':8, 'Century':16, 'Duck':-2, '170sr':6,
                 '150sr':4, '130sr':2, '70sr':-2, '60sr':-4, '50sr':-6}

Bowling_points = {'Wicket':25, 'LBW_Bowled':8, '3W':4, '4W':8, 
                  '5W':16, 'Maiden':12, '5rpo':6, '6rpo':4, '7rpo':2, '10rpo':-2,
                 '11rpo':-4, '12rpo':-6}

Fielding_points = {'Catch':8, '3Cath':4, 'Stumping':12, 'RunOutD':12,
                  'RunOutInd':6}

Checking performance of batsman in one team against another team's bowlers

In [ ]:
team1 = mi; team2 = rcb

for i in range(len(team1)):
    ffp = []
    for j in range(len(team2)):
        bat_vs_bowl = byb[(byb["batsman"]==team1[i]) & (byb["bowler"]==team2[j])]
        bowls_played = len(bat_vs_bowl.batsman_runs)
        runs_scored = sum(bat_vs_bowl.batsman_runs)
        fours = len(bat_vs_bowl[bat_vs_bowl['batsman_runs']==4])
        sixes = len(bat_vs_bowl[bat_vs_bowl['batsman_runs']==6])
        wicket = sum(bat_vs_bowl.is_wicket)
        if bowls_played <=6*10 and wicket >=5:
            penalty = -16
            print (team1[i], "ka wicket taken",wicket,"times by", team2[j])
        elif bowls_played <=6*8 and wicket >=4:
            penalty = -8
            print (team1[i], "ka wicket taken",wicket,"times by", team2[j])
        elif bowls_played <=6*6 and wicket >=3:
            penalty = -4
            print (team1[i], "'s wicket taken",wicket,"times by", team2[j])
        else:
            penalty = 0
        try:    
            strike_rate = int(runs_scored/bowls_played*100)
        except: 
            strike_rate = 'NA'

        if bowls_played >=8 and strike_rate!='NA':
            if strike_rate >=170:
                print (team1[i] ,"beaten", team2[j], "Runs", runs_scored,"bowls",bowls_played,"strike rate", strike_rate,'Out',wicket,'times', "Fours", fours,"Sixes", sixes)            
            elif strike_rate >=150:
                print (team1[i] ,"beaten", team2[j], "Runs", runs_scored,"bowls",bowls_played,"strike rate", strike_rate,'Out',wicket,'times', "Fours", fours,"Sixes", sixes)            
                        
        bowl_vs_bat = byb[(byb["bowler"]==team1[i]) & (byb["batsman"]==team2[j])]
        wicket_took = sum(bowl_vs_bat.is_wicket)
        fantasy_points1 = runs_scored + fours*Batsman_points['bFour'] + sixes*Batsman_points['bSix'] - wicket*Bowling_points['Wicket'] + wicket_took*Bowling_points['Wicket'] + penalty
        ffp.append(fantasy_points1)
#         print (team1[i] ,"against", team2[j], "Runs", runs_scored, 
#                "bowls",bowls_played,"strike rate", strike_rate,
#               'Out',wicket,'times', "Fours", fours,"Sixes", sixes)

taking other fantasy points into consideration

In [ ]:
def get_players(team1,team2,team1_fp):
    fantasy_team_players = []

    for i in range(len(team1)):
        unq_ids = byb[byb["batsman"]==team1[i]]['id'].unique()
        mathces_played = len(unq_ids)
#         print ( "Number of matches played" , len(unq_ids),team1[i])
        bbr = []
        for x in unq_ids:
            bat_run = sum(byb[(byb["batsman"]==team1[i])&(byb['id']==x)]['batsman_runs'])
            bbr.append(bat_run)

        r30,r50,r100 =0,0,0
        for m in bbr:
            if m>=100:
                r100+=1
            elif m>=50:
                r50+=1
            elif m>=30:
                r30+=1
        try:
            catches = len(byb[(byb['fielder']==team1[i]) & (byb['dismissal_kind']=='caught')])/mathces_played
            run_outs = len(byb[(byb['fielder']==team1[i]) & (byb['dismissal_kind']=='run out')])/mathces_played
            extra_points = r30/mathces_played*Batsman_points['30Runs'] +r50/mathces_played*Batsman_points['Half_century'] +r100/mathces_played*Batsman_points['Century'] +catches*Fielding_points['Catch']+run_outs*Fielding_points['RunOutInd']
        except:
            catches, run_outs, extra_points = 0,0,0
        
        # Extra Points for bowlers to be estimated here
        wickets_taken = []
        for x in unq_ids:
            twx = sum(byb[(byb["bowler"]==team1[i]) & (byb['id']==x)]['is_wicket'])
            wickets_taken.append(twx)

        w3,w4,w5 = 0,0,0
        for z in wickets_taken:
            if z>=5:
                w5+=1
            elif z>=4:
                w4+=1
            elif z>=3:
                w3+=1
        try:
            lbws = len((byb[(byb['bowler']==team1[i]) & (byb['dismissal_kind']=='lbw')]))/mathces_played      
            bowled = len((byb[(byb['bowler']==team1[i]) & (byb['dismissal_kind']=='bowled')]))/mathces_played      
            wexp = w3/mathces_played*Bowling_points['3W'] + w4/mathces_played*Bowling_points['4W'] + w5/mathces_played*Bowling_points['5W'] + lbws*Bowling_points['LBW_Bowled'] + bowled*Bowling_points['LBW_Bowled']
        except:
            lbws, bowled, wexp = 0,0,0
        
        ffp = []
        for j in range(len(team2)):
            bat_vs_bowl = byb[(byb["batsman"]==team1[i]) & (byb["bowler"]==team2[j])]
            bowls_played = len(bat_vs_bowl.batsman_runs)
            runs_scored = sum(bat_vs_bowl.batsman_runs)
            fours = len(bat_vs_bowl[bat_vs_bowl['batsman_runs']==4])
            sixes = len(bat_vs_bowl[bat_vs_bowl['batsman_runs']==6])
            wicket = sum(bat_vs_bowl.is_wicket)
            if bowls_played <=6*10 and wicket >=5:
                penalty = -16
                print (team1[i], "ka wicket taken",wicket,"times by", team2[j])
            elif bowls_played <=6*8 and wicket >=4:
                penalty = -8
                print (team1[i], "ka wicket taken",wicket,"times by", team2[j])
            elif bowls_played <=6*6 and wicket >=3:
                penalty = -4
                print (team1[i], "'s wicket taken",wicket,"times by", team2[j])
            else:
                penalty = 0

            try:    
                strike_rate = int(runs_scored/bowls_played*100)
            except: 
                strike_rate = 'NA'            
            if bowls_played >=10 and strike_rate!='NA':
                if strike_rate >=170:
                    print (team1[i] ,"beaten", team2[j], "Runs", runs_scored,"bowls",bowls_played,"strike rate", strike_rate,'Out',wicket,'times', "Fours", fours,"Sixes", sixes)            
                elif strike_rate >=150:
                    print (team1[i] ,"beaten", team2[j], "Runs", runs_scored,"bowls",bowls_played,"strike rate", strike_rate,'Out',wicket,'times', "Fours", fours,"Sixes", sixes)            

                
            bowl_vs_bat = byb[(byb["bowler"]==team1[i]) & (byb["batsman"]==team2[j])]
            wicket_took = sum(bowl_vs_bat.is_wicket)
            fantasy_points1 = runs_scored + fours*Batsman_points['bFour'] + sixes*Batsman_points['bSix'] - wicket*Bowling_points['Wicket'] + wicket_took*Bowling_points['Wicket'] + penalty 
            ffp.append(fantasy_points1)
#             print (team1[i] ,"against", team2[j], "Runs", runs_scored, 
#                    "bowls",bowls_played,"strike rate", strike_rate,
#                   'Out',wicket,'times', "Fours", fours,"Sixes", sixes, "fatansy points",fantasy_points1)
        sum_ffp = sum(ffp)
        if team1_fp[team1[i]] > 0:
            recent_performace_points = np.log(team1_fp[team1[i]])
        elif team1_fp[team1[i]] <0:
            recent_performace_points = -np.log(abs(team1_fp[team1[i]]))
        else:
            recent_performace_points = 0
        # Trying a new method for recent performancec point
        recent_performace_points = team1_fp[team1[i]]/3
        weight1 = 0.5
        weight2 = 1 - weight1
        final_fantasy_point = (sum_ffp + extra_points + wexp)*weight1 + recent_performace_points*weight2
        final_fantasy_point = round(final_fantasy_point,2)
        fantasy_team_players.append((final_fantasy_point,team1[i]))
        fantasy_team_players.sort(reverse=True)
#         print ("Fatasy points of",team1[i],final_fantasy_point)
    return fantasy_team_players

t1 = get_players(mi, rcb, mi_fp)
print ('\n')
t2 = get_players(rcb, mi, rcb_fp)
t3 = t1 + t2
t3.sort(reverse = True)
print ("================================================\n")
print ("Team 1", t1)
print ("================================================\n")
print ("Team 2", t2)
print('\n Your final team')
t3[:15]

https://www.kaggle.com/code/malaydhami/fantasy-ipl-dream-team/notebook
Batting Points
Run -> +1

Boundary Bonus -> +1

Six Bonus -> +2

30 Run Bonus -> +4

Half-century Bonus -> +8

Century Bonus -> +16
finds bat_score according to point system mentioned above.

In [ ]:
def batsman_score(x):
    x["bat_score"]=1*x["runs"]+1*x["fours"]+2*x["sixes"]+4*x["thirty"]+8*x["fifty"]+16*x["century"]
    return x

def get_bat_stats(df):
    # Initialize batting dataframe.
    batting_df = pd.DataFrame(columns=["player","runs","fours","sixes","thirty","fifty","century"])

    # Let's use groupby function. Our aim is to find batting score so we'll group our data by batsman.
    for player,details in df.groupby("batsman"):
        total_runs = sum(details["batsman_runs"])

        # If batsman_runs is 4 then we count it as four.
        fours=0
        try:
            fours=details["batsman_runs"].value_counts()[4]
        except:
            fours=0      

        # If batsman_runs is 6 then we count it as six.
        sixes=0
        try:
            sixes=details["batsman_runs"].value_counts()[6]
        except:
            sixes=0 

        thirty=0
        fifty=0
        century=0

        # To find thirty, fifty, century of the player we need to group our dataframe(details) by match id.
        for match,match_details in details.groupby("id"):

            # Find total score of the player by sum up total runs.
            match_score = sum(match_details["batsman_runs"])

            # Accourding to match_score this count thirties, fifties, centuries.
            if match_score>=100:
                century+=1
            elif match_score>=50:
                fifty+=1
            elif match_score>=30:
                thirty+=1

        # Store all the data into batting dataframe.
        batting_df = batting_df.append({"player":player,"runs":total_runs,"fours":fours,"sixes":sixes,"thirty":thirty,"fifty":fifty,"century":century},ignore_index=True)
    
    # To find bat_score we apply batsman_score function
    batting_df=batting_df.apply(batsman_score,axis=1)
    return batting_df

Bowling Points
Wicket -> +25

Bonus (LBW / Bowled) -> +8

3 Wicket Bonus -> +4

4 Wicket Bonus -> +8

5 Wicket Bonus -> +16

In [ ]:
def bowler_score(x):
    x["bowl_score"]=25*x["wickets"]+8*x["lbw_or_bowled"]+4*x["three_wickets"]+8*x["four_wickets"]+16*x["five_wickets"]
    return x

def get_bowl_stats(df):
    # Initialize bowling dataframe.
    bowling_df = pd.DataFrame(columns=["player","wickets","lbw_or_bowled","three_wickets","four_wickets","five_wickets"])

    # Let's use groupby function. Our aim is to find bowling score so we'll group our data by bowler.
    for player,details in df.groupby("bowler"):

        # It finds the length of dataframe where dismissal kind is either lbw or bowled
        lbw_or_bowled=len(details.query('dismissal_kind == "lbw" or dismissal_kind == "bowled"'))

        total_wickets=0
        three_wickets=0
        four_wickets=0
        fifer=0

        # To find three wicket haul, four wicket haul and five wicket haul we should group dataframe(details) again by match id.
        for match,match_details in details.groupby("id"):
            wickets = len(match_details.query('dismissal_kind in ["caught","bowled","lbw","stumped","caught and bowled"]'))
            if wickets>=3:
                three_wickets+=1
            elif wickets>=4:
                four_wickets+=1
            elif wickets>=5:
                fifer+=1
            # Need to add every match wickets into total_wickets to find total wickets of bowler.
            total_wickets+=wickets

        # Store all the data into bowling dataframe.
        bowling_df = bowling_df.append({"player":player,"wickets":total_wickets,"lbw_or_bowled":lbw_or_bowled,"three_wickets":three_wickets,"four_wickets":four_wickets,"five_wickets":fifer},ignore_index=True)
        
    # To find bowl_score we apply bowler_score function.
    bowling_df=bowling_df.apply(bowler_score,axis=1) 
    return bowling_df

In [ ]:
def get_wicketkeepers(df):
    # Let's find out wicketkeepers from this dataset. To achieve this we'll check dismissal kind, if it is stumpout then store such fielders name into list
    return list(df.query('dismissal_kind == "stumped"')['fielder'].unique())